In [ ]:
# pip install tape-proteins

In [ ]:
#HuggingFace Tranformers Package
# pip install transformers

In [ ]:
# pip install keras

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from scipy.stats import norm
import scipy.stats as sps
import cv2
from scipy.ndimage import convolve
import skimage
from scipy import ndimage
from scipy.spatial.distance import cdist
from nibabel.testing import data_path
import nibabel as nib
from sklearn.mixture import GaussianMixture
import os
import pandas as pd
import skimage.measure
import seaborn as sns
import scipy as sp
import gudhi as gd
import random
import sys
sys.path.append('../../') 
from Code.function_geometry import * 
from Code.codegeometry import * 
from PIL import Image
from sklearn.utils import shuffle

In [ ]:
import sklearn.preprocessing   as skp
import sklearn.neighbors       as skn
import sklearn.model_selection as skm
import sklearn.decomposition   as skd
import sklearn.manifold        as skf
import sklearn.pipeline        as skl
import sklearn.svm             as sks
import sklearn.ensemble        as ske
import itertools
import tensorflow as tf

import gudhi.clustering.tomato as gdt
import gudhi.representations   as gdr

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
#tokenizer
from tape import ProteinBertModel, TAPETokenizer
# from tape import UniRepModel, UniRepTokenizer
# from tape import ProteinBertModel, ProteinBertTokenizer
from transformers import AutoTokenizer
from transformers import BertModel, BertTokenizer
# ProtGPT2
# from transformers import pipeline

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D,Conv2D, Dense, Flatten, Dropout,MaxPooling2D
from keras import layers
from keras.layers import MaxPooling1D,MaxPooling2D
from keras.optimizers import SGD, RMSprop
from keras import regularizers

In [ ]:
def createFaster(filename,labels,sequences):
    with open(filename, 'w') as f:
        # Write content to the file
        for i in range(len(sequences)):
        # for i in range(5):
            f.write(">"+str(labels[i]))
            f.write(sequences[i]+"\n")

In [ ]:

def signalProtein(filename,sequences,labels):
    new_label = []
    new_sequence = []
    i = 0
    with open(filename, 'w') as f:
        for r in result_signalp:
            if r > 0 and r < 0.7:
                new_label.append(0)
            elif r > 0.7:
                new_label.append(1)
            if r > 0:
                f.write(labels[i]+"\n")
                f.write(sequences[i]+"\n")
                new_sequence.append(sequences[i])
            i+=1
    return new_label,new_sequence         

In [ ]:
def resultSignalP(filename):
    f=open(filename,'r')
    result=[]
    next(f)
    next(f)
    for line in f.readlines():
        data = line.rstrip()  
        index = data.__contains__('SP')
        #Can we return the probabilty of SP and other
        if index == True: 
            result.append(1)
        else:
            result.append(0)
    f.close()
    return result  

In [ ]:
def geotop_analysis(data):
    # U_train = U_train_b + U_train_m + U_test_b + U_test_m
    n = 100
    i = 0
    dgms_tda = []
    dgms_per_var = []
    dgms_per_linalg = []
    for U in data:
        print(i)
        L = np.linspace(np.min(U), np.max(U),  n)[::-1]
        life, barecode, persistence, connected_comp, Per_total, Area_total, euler_total, per, area, euler = function_persistance(U, L, False)
        diagram_pondere_total_vari = np.zeros((len(life), 2))
        diagram_pondere_linalg = np.zeros((len(life), 2))
        diagram = np.zeros((len(life), 2))
        k = 0
        idx_keys = list(life.keys())
        for idx in idx_keys[1:]:
            diagram_pondere_total_vari[k][0] = (np.sum(per[idx]))*life[idx][1]
            diagram_pondere_total_vari[k][1] = (np.sum(per[idx]))*life[idx][0]
            diagram_pondere_linalg[k][0] = (np.linalg.norm(per[idx]))*life[idx][1]
            diagram_pondere_linalg[k][1] = (np.linalg.norm(per[idx]))*life[idx][0]
            diagram[0] = life[idx][1]
            diagram[1] = life[idx][0]
            k=k+1
        dgms_tda.append(diagram)
        dgms_per_linalg.append(diagram_pondere_linalg)
        dgms_per_var.append(diagram_pondere_total_vari)
        i=i+1
    return dgms_per_linalg

In [ ]:
def concate_embedding_geotop(embedding,dgms,dim):
    long = embedding.shape[1]
    res = []
    i = 0
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest withe the remainning zeroes
        t = dim - (long + flat.shape[0])
        res.append(np.concatenate((np.concatenate((embedding[i], flat)), np.zeros(t))))
        i +=1
    return res

In [ ]:
def model_trainning_saving(shape,epoch,model_name,train_data,train_labels,test_data,test_labels):
    model = Sequential()

    model.add(Conv2D(filters=8, kernel_size=(3, 3), padding="same",activation='relu', input_shape=(shape[0],shape[1],1)))
    model.add(layers.AveragePooling2D())
    
    model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), padding="same",activation='relu'))
    model.add(layers.AveragePooling2D())
    
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(layers.AveragePooling2D())
    
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(units=128, activation='relu',kernel_regularizer=regularizers.l2(1e-4),bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)))
    
    model.add(layers.Dense(units=64, activation='relu',kernel_regularizer=regularizers.l2(1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
    
    model.add(layers.Dense(units=1, activation = 'sigmoid',kernel_regularizer=regularizers.l2(1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    print(model.summary())

    #trainning
    history = model.fit(x=train_data, y=train_labels,epochs=epoch,shuffle=True,validation_data=(test_data, test_labels),verbose=1,batch_size=64)
  
    
    # Plot history: MAE
    import matplotlib.pyplot as plt
    plt.plot(history.history['accuracy'], label='training data')
    plt.plot(history.history['val_accuracy'], label='validation data')
    #plt.title('')
    plt.ylabel('Accuracy')
    plt.xlabel('No. epoch')
    plt.legend(loc="lower right")
    plt.show()
    model.save(model_name+'.h5')
  
    return model

In [ ]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

In [ ]:
def concate_embedding_geotop(embedding,dgms,dim):
    # len_data = max(array.shape[0] for array in dgms)
    long = embedding.shape[1]
    res = []
    i = 0
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest with zeroes
        t = dim - (long + flat.shape[0])
        res.append(np.concatenate((np.concatenate([embedding[i], flat]), np.zeros(t))))
        i +=1
    return res

In [ ]:
def concat_embedding_geotop(dgms,dim):
    # len_data = max(array.shape[0] for array in dgms)
    res = []
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest with zeroes
        t = dim - flat.shape[0]
        res.append(np.concatenate([flat, np.zeros(t)]))
    return res

In [ ]:
def sp_no_sp(label):
    if label == "NO_SP":
        return 0
    else:
        return 1

In [ ]:
def readFastaFiles(filename):
    sequences = []
    labels = []
    with open(filename, 'r') as file:
        current_id = None
        current_seq = ''
        skip_line = False
        for line in file:
            # print("*"*10)
            # print(line)
            if not skip_line:
                if line.startswith('>'):
                    if current_id:
                        # print(sp_no_sp(current_id.strip()[1:].split("|")[2]),current_id.strip()[1:].split("|")[2])
                        labels.append(sp_no_sp(current_id.strip()[1:].split("|")[2]))
                        sequences.append(current_seq.replace('\n', ''))
                        # skip_line = not skip_line
                    current_id = line
                    current_seq = ''
                else:
                    current_seq += line
                    skip_line = not skip_line
            else:
                skip_line = not skip_line
        # Add the last sequence
        if current_id:
            labels.append(sp_no_sp(current_id.strip()[1:].split("|")[2]))
            sequences.append(current_seq.replace('\n', ''))
    return labels,sequences

In [ ]:
label,sequence=readFastaFiles("dataset.fasta")  # change to your own path

In [ ]:
len(sequence)

In [ ]:
pd.DataFrame(label)[0].unique()

In [ ]:
pd.value_counts(pd.DataFrame(label)[0])

In [ ]:
len(label)

## Data Manipulation

In [ ]:
#Creating fasta file with protein <=200
createFaster("all_data.fasta",label,sequence)

In [ ]:
new_label = label.copy()

In [ ]:
len(new_label)

### Embedding using TAPE

In [ ]:
#Learning the representation of protein
model_tape = ProteinBertModel.from_pretrained('bert-base')
tokenizer_tape = TAPETokenizer(vocab='iupac')

In [ ]:
num_of_features = 768
embedded=np.zeros((len(new_label),num_of_features))
y=np.zeros(len(new_label))

i=0
for s in sequence:
    #tape
    print(i)
    token_ids = torch.tensor([tokenizer_tape.encode(s)])
    output = model_tape(token_ids)
    sequence_output = output[0]
    embedded[i]= np.array(np.mean(sequence_output.detach().numpy(),axis=1) )
    i=i+1

In [ ]:
# Save the array to a file
# np.save('alldata_tape.npy', embedded)

# # Reload the array
embedded = np.load('alldata_tape.npy')

### Embedding using ESM

In [ ]:
protein_data = [("protein_" + str(i), s) for i, s in enumerate(sequence)]

In [ ]:
#embedding 2
import torch
import esm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

def get_average_embedding(sequence):
    _, _, batch_tokens = batch_converter(sequence)
    # Get the embeddings
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    # Compute the average embedding
    avg_embedding = torch.mean(results["representations"][33], dim=1)  # Assuming layer 33 for representation
    return avg_embedding


In [ ]:

embedding2 = []
i = 0
for s in protein_data:
    print(i)
    embedding2.append(get_average_embedding([s]))
    i+=1

In [ ]:
# embedding2

In [ ]:
# Save the tensor to a file
# torch.save(embedding2, 'alldata_esm.pt')

# Load the tensor from the file
embedding2 = torch.load('alldata_esm.pt')

In [ ]:
# Convert each tensor to a NumPy array
numpy_array_list = [tensor.numpy() for tensor in embedding2]

# Convert the list of arrays to a single NumPy array
numpy_array = np.array(numpy_array_list)

In [ ]:
numpy_array = numpy_array.reshape(numpy_array.shape[0],numpy_array.shape[2])

In [ ]:
numpy_array.shape

### Converting embedding to image

In [ ]:
embedding_padded = np.concatenate((embedded, np.zeros((embedded.shape[0],16))), axis=1)
embedding2_padded = np.concatenate((numpy_array, np.zeros((numpy_array.shape[0],16))), axis=1)

In [ ]:
np.save('alldata_tape_scaled.npy', embedding_padded)
np.save('alldata_esm_scaled.npy', embedding2_padded)

In [ ]:
embedding2_padded.shape

In [ ]:
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(embedding_padded)
embedding_normalized = std_scale.transform(embedding_padded)

In [ ]:
std_scale = preprocessing.StandardScaler().fit(embedding2_padded)
embedding2_normalized = std_scale.transform(embedding2_padded)

In [ ]:
print(embedding2_normalized[0])

In [ ]:
#first image
shape_unit=28
embedding_images=np.reshape(embedding_normalized,(embedding_normalized.shape[0],shape_unit,shape_unit))

print(embedding_images.shape)

In [ ]:
embedding_images[0].shape

In [ ]:
#first image
shape_unit=36
embedding2_images=np.reshape(embedding2_normalized,(embedding2_normalized.shape[0],shape_unit,shape_unit))

# print(embedding2_images[0])

### Getting the geometrical representation of each embedded image

In [ ]:
#apply geotop to the first image embedding
#28*28
embedding_geotop = geotop_analysis(embedding_images)

In [ ]:
36*36
embedding2_geotop = geotop_analysis(embedding2_images)

In [ ]:
embedding2_geotop[0]

In [ ]:
np.savez("alldata_tape_geotop.npz", *embedding_geotop)

In [ ]:
np.savez("alldata_esm_geotop.npz", *embedding2_geotop)

In [ ]:
# # Load list of NumPy arrays from file
loaded_arrays = np.load("alldata_tape_geotop.npz")
loaded_arrays2 = np.load("alldata_esm_geotop.npz")

# # Access individual arrays using their keys
# array1 = loaded_arrays['arr_0']
# array2 = loaded_arrays['arr_1']

In [ ]:
embedding_geotop = [loaded_arrays['arr_'+str(i)] for i in range(len(loaded_arrays))]

In [ ]:
embedding2_geotop = [loaded_arrays['arr_'+str(i)] for i in range(len(loaded_arrays2))]

## Trainning

### TAPE and Geotop

In [ ]:
dgms = gdr.DiagramSelector(use=True, point_type='finite').fit_transform(embedding_geotop)

In [ ]:
len(dgms)

In [ ]:
dim = 1024
result = concate_embedding_geotop(embedded,dgms,dim)

In [ ]:
#to be verify
# shape_unit=32
# tape_geotope_result = np.reshape(result,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
len(result)

In [ ]:
tape_geotope_result = result.copy()

In [ ]:
#splitting the dataset
num_samples = len(result)
num_train_samples = int(0.8 * num_samples)  # 80% for training, 20% for testing

# Split data and labels into training and testing sets
train_data, test_data = result[:num_train_samples], result[num_train_samples:]
train_labels, test_labels = label[:num_train_samples], label[num_train_samples:]

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
shape_unit=32
std_scale = preprocessing.StandardScaler().fit(train_data)
X_normalized = std_scale.transform(train_data)
train_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
std_scale = preprocessing.StandardScaler().fit(test_data)
X_normalized = std_scale.transform(test_data)
test_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
# std_scale = preprocessing.StandardScaler().fit(test_data)
np.save('alldata_tape_scaled_test.npy', test_data)

In [ ]:
# test_data

In [ ]:
test_data[0].shape

In [ ]:
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)

In [ ]:
train_data[0].shape

In [ ]:
unique, counts = np.unique(test_labels, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
test_labels.shape

In [ ]:
test_data.shape

In [ ]:
train_data.shape

In [ ]:
for i in range(30):
    plt.matshow(train_data[i])
    plt.title(train_labels[i])
    plt.show()

In [ ]:
 temp_model = model_trainning_saving([32,32],50,"alldata_tape_model",train_data,train_labels,test_data,test_labels)

In [ ]:
temp_result = temp_model.predict(train_data)

### ESM and Geotop

In [ ]:
sequence[1]

In [ ]:
embedding2[1]

In [ ]:
dgms = gdr.DiagramSelector(use=True, point_type='finite').fit_transform(embedding2_geotop)

In [ ]:
dim = 1681
result = concate_embedding_geotop(numpy_array,dgms,dim)

In [ ]:
embedding2[0]

In [ ]:
result[0]

In [ ]:
#splitting the dataset
num_samples = len(result)
num_train_samples = int(0.8 * num_samples)  # 80% for training, 20% for testing

# Split data and labels into training and testing sets
train_data, test_data = result[:num_train_samples], result[num_train_samples:]
train_labels, test_labels = label[:num_train_samples], label[num_train_samples:]

In [ ]:
np.save('alldata_esm_scaled_test.npy', test_data)

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
shape_unit=41
std_scale = preprocessing.StandardScaler().fit(train_data)
X_normalized = std_scale.transform(train_data)
train_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
std_scale = preprocessing.StandardScaler().fit(test_data)
X_normalized = std_scale.transform(test_data)
test_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
np.save('alldata_esm_scaled_test.npy', test_data)

In [ ]:
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)

In [ ]:
unique, counts = np.unique(train_labels, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
train_data.shape

In [ ]:
for i in range(30):
    plt.matshow(train_data[i])
    plt.title(train_labels[i])
    plt.show()

In [ ]:
 rest = model_trainning_saving([41,41],45,"esm_model_alldata_new",train_data,train_labels,test_data,test_labels)

In [ ]:
temp_result = rest.predict(train_data)

In [ ]:
i = 1
for u in temp_result:
    if u>0.5:
        print("*** ",i," *** ",train_labels[i])
        print(round(u[0],4))
    i = i+1
print("***************")
print(i)

In [ ]:
pd.value_counts(pd.DataFrame(train_labels)[0])

### TAPE

In [ ]:
result = np.concatenate((embedded, np.zeros((embedded.shape[0],16))), axis=1)

In [ ]:
#splitting the dataset
num_samples = len(result)
num_train_samples = int(0.8 * num_samples)  # 80% for training, 20% for testing

# Split data and labels into training and testing sets
train_data, test_data = result[:num_train_samples], result[num_train_samples:]
train_labels, test_labels = label[:num_train_samples], label[num_train_samples:]

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
shape_unit=28
std_scale = preprocessing.StandardScaler().fit(train_data)
X_normalized = std_scale.transform(train_data)
train_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
std_scale = preprocessing.StandardScaler().fit(test_data)
X_normalized = std_scale.transform(test_data)
test_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)

In [ ]:
unique, counts = np.unique(train_labels, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
train_data.shape

In [ ]:
for i in range(30):
    plt.matshow(train_data[i])
    plt.title(train_labels[i])
    plt.show()

In [ ]:
 model_trainning_saving([28,28],45,"tape_only_model",train_data,train_labels,test_data,test_labels)

### Geotop of TAPE

In [ ]:
dgms = gdr.DiagramSelector(use=True, point_type='finite').fit_transform(embedding_geotop)

In [ ]:
#15*15
dim = 225
result = concat_embedding_geotop(dgms,dim)

In [ ]:
#splitting the dataset
num_samples = len(result)
num_train_samples = int(0.8 * num_samples)  # 80% for training, 20% for testing

# Split data and labels into training and testing sets
train_data, test_data = result[:num_train_samples], result[num_train_samples:]
train_labels, test_labels = label[:num_train_samples], label[num_train_samples:]

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
shape_unit=15
std_scale = preprocessing.StandardScaler().fit(train_data)
X_normalized = std_scale.transform(train_data)
train_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
std_scale = preprocessing.StandardScaler().fit(test_data)
X_normalized = std_scale.transform(test_data)
test_data=np.reshape(X_normalized,(X_normalized.shape[0],shape_unit,shape_unit))

In [ ]:
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)

In [ ]:
for i in range(30):
    plt.matshow(train_data[i])
    plt.title(train_labels[i])
    plt.show()

In [ ]:
 model_trainning_saving([15,15],45,"geotop_of_tape_model",train_data,train_labels,test_data,test_labels)